In [1]:
import numpy as np

In [12]:
int(5/3)
#np.round(2.4)

1

In [13]:
np.random.seed(2)

In [52]:
import pandas as pd
import featuretools as ft
import featuretools.variable_types as vtypes
#### create dfs
d = {'ATTEN_ID': [1, 2, 3], 
     'PATIENT_ID': [1, 1, 2], 
     'ARRIVE_TIME': ['2018-01-01 01:00:00', '2018-01-01 02:00:00', '2018-01-01 04:00:00'],
     'DEPART_TIME': ['2018-01-01 01:30:00', '2018-01-01 04:30:00', '2018-01-01 06:15:00'],
    'TOTAL_TIME':[0.5,2.5,2.25]}
df = pd.DataFrame(data=d)

d = {'HOUR':['2018-01-01 00:00:00','2018-01-01 01:00:00','2018-01-01 02:00:00','2018-01-01 03:00:00','2018-01-01 04:00:00','2018-01-01 05:00:00','2018-01-01 06:00:00']}
df_hour = pd.DataFrame(d)

data_variable_types = {'ATTEN_ID': vtypes.Id,
                       'PATIENT_ID': vtypes.Id,
                       'ARRIVE_TIME': vtypes.Datetime,
                       'DEPART_TIME': vtypes.Datetime,
                       'TOTAL_TIME': vtypes.Numeric}

d = {'ATTEN_ID': [1, 2,2,2, 3,3,3],
    'HOUR': ['2018-01-01 01:00:00',
                    '2018-01-01 02:00:00','2018-01-01 03:00:00','2018-01-01 04:00:00',
                    '2018-01-01 04:00:00','2018-01-01 05:00:00','2018-01-01 06:00:00']}

df_active_visits = pd.DataFrame(d) # a many-many list of active visits to hosp

#### make ESets
es = ft.EntitySet('Hospital')
# Make attendances
es = es.entity_from_dataframe(entity_id='attendances',
                               dataframe=df,
                               index='ATTEN_ID',
                               time_index='ARRIVE_TIME',
                              secondary_time_index={'DEPART_TIME':['TOTAL_TIME']},
                               variable_types=data_variable_types)
# Make patients
es.normalize_entity(base_entity_id='attendances',
                   new_entity_id='patients',
                   index='PATIENT_ID',
                   make_time_index=False)

# Make linking-es (active_visits)
es = es.entity_from_dataframe(entity_id='active_visits',
                               dataframe=df_active_visits,
                              make_index=True,
                              index='index',
                               variable_types={'HOUR':vtypes.Datetime})

# Make hours eset
es = es.entity_from_dataframe(entity_id='hours',
                               dataframe=df_hour,
                               index='HOUR',
                               variable_types={'HOUR':vtypes.Datetime})
# add es relationships
rel_Atten_ActiveVisits = ft.Relationship(es["attendances"]["ATTEN_ID"],
                                    es["active_visits"]["ATTEN_ID"])
rel_Hours_ActiveVisits = ft.Relationship(es["hours"]["HOUR"],
                                    es["active_visits"]["HOUR"])
es = es.add_relationships([rel_Atten_ActiveVisits,rel_Hours_ActiveVisits]) 


fm, features = ft.dfs(entityset=es,
                     target_entity='hours',
                     agg_primitives=['mean','count'],
                     trans_primitives=[],
                     verbose=True,
                     max_depth = 4)
fm

Built 6 features
Elapsed: 00:00 | Remaining: 00:00 | Progress: 100%|███████████| Calculated: 1/1 chunks


,COUNT(active_visits),MEAN(active_visits.attendances.TOTAL_TIME),MEAN(active_visits.attendances.COUNT(active_visits)),MEAN(active_visits.attendances.patients.MEAN(attendances.TOTAL_TIME)),MEAN(active_visits.attendances.patients.COUNT(attendances)),MEAN(active_visits.attendances.patients.COUNT(active_visits))
HOUR,,,,,,
2018-01-01 00:00:00,0.0,NaN,NaN,NaN,NaN,NaN
2018-01-01 01:00:00,1.0,0.500,1.0,1.500,2.0,4.0
2018-01-01 02:00:00,1.0,2.500,3.0,1.500,2.0,4.0
2018-01-01 03:00:00,1.0,2.500,3.0,1.500,2.0,4.0
2018-01-01 04:00:00,2.0,2.375,3.0,1.875,1.5,3.5
2018-01-01 05:00:00,1.0,2.250,3.0,2.250,1.0,3.0
2018-01-01 06:00:00,1.0,2.250,3.0,2.250,1.0,3.0


In [49]:
es['active_visits'].df

,index,ATTEN_ID,HOUR
0,0,1,2018-01-01 01:00:00
1,1,2,2018-01-01 02:00:00
2,2,2,2018-01-01 03:00:00
3,3,2,2018-01-01 04:00:00
4,4,3,2018-01-01 04:00:00
5,5,3,2018-01-01 05:00:00
6,6,3,2018-01-01 06:00:00


In [29]:
es

Entityset: Hospital
  Entities:
    attendances [Rows: 3, Columns: 5]
    patients [Rows: 2, Columns: 1]
    active_visits [Rows: 7, Columns: 3]
    hours [Rows: 6, Columns: 1]
  Relationships:
    attendances.PATIENT_ID -> patients.PATIENT_ID
    active_visits.ATTEN_ID -> attendances.ATTEN_ID
    active_visits.HOUR -> hours.HOUR

In [30]:
es['attendances'].df

,ATTEN_ID,PATIENT_ID,ARRIVE_TIME,DEPART_TIME,TOTAL_TIME
1,1,1,2018-01-01 01:00:00,2018-01-01 01:30:00,0.50
2,2,1,2018-01-01 02:00:00,2018-01-01 04:30:00,2.50
3,3,2,2018-01-01 04:00:00,2018-01-01 06:15:00,2.25


In [31]:
es['patients'].df

,PATIENT_ID
1,1
2,2


In [32]:
es['hours'].df

,HOUR
2018-01-01 01:00:00,2018-01-01 01:00:00
2018-01-01 02:00:00,2018-01-01 02:00:00
2018-01-01 03:00:00,2018-01-01 03:00:00
2018-01-01 04:00:00,2018-01-01 04:00:00
2018-01-01 05:00:00,2018-01-01 05:00:00
2018-01-01 06:00:00,2018-01-01 06:00:00


In [33]:
es['active_visits'].df

,index,ATTEN_ID,HOUR
0,0,1,2018-01-01 01:00:00
1,1,2,2018-01-01 02:00:00
2,2,2,2018-01-01 03:00:00
3,3,2,2018-01-01 04:00:00
4,4,3,2018-01-01 04:00:00
5,5,3,2018-01-01 05:00:00
6,6,3,2018-01-01 06:00:00
